In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 991.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 28.3 MB/

In [ ]:
# Imports for the RAG Model
import torch
import chromadb

from transformers import AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_community.document_loaders.pdf import PDFPlumberLoader
from langchain_text_splitters.character import CharacterTextSplitter
from langchain_text_splitters.base import TokenTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# Load the pdf
from langchain.vectorstores import Chroma

# Imports for the Document Processing
import os
from docx import Document
import fitz
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain.document_loaders import DirectoryLoader

# Install necessary packages
# Web Scrapper
import requests
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import urllib.parse

# For Querying
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFacePipeline

In [ ]:
EMB_INSTRUCTOR_XL = "hkunlp/instructor-xl"
EMB_SBERT_MPNET_BASE = "sentence-transformers/all-mpnet-base-v2"

# Models
LLM_FLAN_T5_XXL = "google/flan-t5-xxl"
LLM_FLAN_T5_XL = "google/flan-t5-xl"
LLM_FASTCHAT_T5_XL = "lmsys/fastchat-t5-3b-v1.0"
LLM_FLAN_T5_SMALL = "google/flan-t5-small"
LLM_FLAN_T5_BASE = "google/flan-t5-base"
LLM_FLAN_T5_LARGE = "google/flan-t5-large"
LLM_FALCON_SMALL = "tiiuae/falcon-7b-instruct"


config = {"persist_directory":'db',
          "load_in_8bit":False,
          "embedding" : EMB_SBERT_MPNET_BASE,
          "llm":LLM_FLAN_T5_BASE,
          }

In [ ]:
def create_sbert_mpnet():
        device = "cuda" if torch.cuda.is_available() else "cpu"
        return HuggingFaceEmbeddings(model_name=EMB_SBERT_MPNET_BASE, model_kwargs={"device": device})


def create_flan_t5_base(load_in_8bit=False):
        # Wrap it in HF pipeline for use with LangChain
        model = "google/flan-t5-base"
        tokenizer = AutoTokenizer.from_pretrained(model)
        return pipeline(
            task="text2text-generation",
            model=model,
            tokenizer = tokenizer,
            max_new_tokens=100,
            model_kwargs={"device_map": "auto", "load_in_8bit": load_in_8bit, "max_length": 512, "temperature": 1}
        )

def create_falcon_instruct_small(load_in_8bit=False):
        model = "tiiuae/falcon-7b-instruct"
        tokenizer = AutoTokenizer.from_pretrained(model)
        return pipeline(
                task="text-generation",
                model = model,
                tokenizer = tokenizer,
                max_new_tokens=100,
                model_kwargs={
                    "device_map": "auto",
                    "load_in_8bit": load_in_8bit,
                    "max_length": 512,
                    "temperature": 0.01,
                    # "torch_dtype":torch.bfloat16,
                    "offload_folder": "/data",
                    "eos_token_id":tokenizer.eos_token_id,
                    }
            )



if config["embedding"] == EMB_SBERT_MPNET_BASE:
    embedding = create_sbert_mpnet()
load_in_8bit = config["load_in_8bit"]
if config["llm"] == LLM_FALCON_SMALL :
    llm = create_falcon_instruct_small(load_in_8bit=load_in_8bit)
if config["llm"] == LLM_FLAN_T5_BASE :
    llm = create_flan_t5_base(load_in_8bit=load_in_8bit)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def scrape_text_from_url(url):
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch the URL: {url}")

    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all('p')
    text = "\n".join([para.get_text() for para in paragraphs])
    return text


# Converts a URL to a valid document name
def url_to_document_name(url):
  parsed_url = urllib.parse.urlparse(url)
  filename = parsed_url.netloc + parsed_url.path
  filename = filename.replace('/', '_')
  filename = filename.replace('.', '_')
  return filename

In [ ]:

url = 'https://en.wikipedia.org/wiki/Indian_independence_movement'
name = url_to_document_name(url)

with open(f"data/{name}.txt", "w") as text_file:
    print(scrape_text_from_url(url), file=text_file)


In [ ]:
def read_text_from_file(file_path):
  loader = DirectoryLoader(file_path, glob="./*.txt", loader_cls=TextLoader)
  documents = loader.load()
  loader = DirectoryLoader(file_path, glob="./*.pdf", loader_cls=PDFPlumberLoader, use_multithreading=True)
  documents.extend(loader.load())
  loader = DirectoryLoader(file_path, glob="./*.docx", loader_cls=Docx2txtLoader)
  documents.extend(loader.load())
  return documents


documents = read_text_from_file('./data')

In [ ]:
# Split documents and create text snippets
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=10, encoding_name="cl100k_base")  # This the encoding for text-embedding-ada-002
texts = text_splitter.split_documents(texts)
print(len(texts))

persist_directory = config["persist_directory"]
chroma_client = chromadb.Client()

vectordb = Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)

116


In [ ]:
hf_llm = HuggingFacePipeline(pipeline=llm)
retriever = vectordb.as_retriever(search_kwargs={"k":4})
qa = RetrievalQA.from_chain_type(llm=hf_llm, chain_type="stuff",retriever=retriever, return_source_documents=True)

In [ ]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
query = "When was the Republic of India made ?"
llm_response = qa(query)
process_llm_response(llm_response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Token indices sequence length is longer than the specified maximum sequence length for this model (598 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1283: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


26 January 1950


Sources:
data/en_wikipedia_org_wiki_Indian_independence_movement.txt
data/en_wikipedia_org_wiki_Indian_independence_movement.txt
data/en_wikipedia_org_wiki_Indian_independence_movement.txt
data/en_wikipedia_org_wiki_Indian_independence_movement.txt


In [ ]:
query = "How much Google credit were we talking about?"
llm_response = qa(query)
process_llm_response(llm_response)

200 or 300,000 worth of Google credit


Sources:
data/Tech Weekly Catchup - 2  (2023-12-30 22_32 GMT+5_30) - Transcript.pdf
data/Tech Weekly Catchup - 2  (2023-12-30 22_32 GMT+5_30) - Transcript.pdf
data/Tech Weekly Catchup - 2  (2023-12-30 22_32 GMT+5_30) - Transcript.pdf
data/Tech Weekly Catchup - 2  (2023-12-30 22_32 GMT+5_30) - Transcript.pdf


In [ ]:
def process_file(file):

    if file.type == "text/plain":
        return file.getvalue().decode("utf-8")
    elif file.type == "application/pdf":
        pdf_reader = PyPDF2.PdfReader(file)
        pdf_text = ""
        for page in pdf_reader.pages:
            pdf_text += page.extract_text()
        return pdf_text
    elif file.type in ["application/msword", "application/vnd.openxmlformats-officedocument.wordprocessingml.document"]:
        doc = docx.Document(file)
        doc_text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
        return doc_text
    else:
        return "Unsupported file type"


In [ ]:
%%writefile app.py
import streamlit as st
#from loaders import process_file
#from RAG import RAGModel
#from langchain.document_loaders import DirectoryLoader


# Initialize RAG model
#rag_model = RAGModel()


def read_text_from_file(file_path):
  loader = DirectoryLoader(file_path, glob="./*.txt", loader_cls=TextLoader)
  documents = loader.load()
  loader = DirectoryLoader(file_path, glob="./*.pdf", loader_cls=PDFPlumberLoader, use_multithreading=True)
  documents.extend(loader.load())
  loader = DirectoryLoader(file_path, glob="./*.docx", loader_cls=Docx2txtLoader)
  documents.extend(loader.load())
  return documents


# Initialize chatbox state in session state
if 'chatbox_open' not in st.session_state:
    st.session_state['chatbox_open'] = False

# Function to add a message to the chat history
def add_chat_message(message):
    if 'messages' not in st.session_state:
        st.session_state['messages'] = []
    st.session_state['messages'].append(message)

def process_documents():
  with st.spinner('Wait for it...'):
      read_text_from_file()

# Sidebar for file uploads
st.sidebar.title("Upload Documentation")
uploaded_files = st.sidebar.file_uploader("Choose files:", accept_multiple_files=True)


# Process button to process files
if st.button("Process Files"):
    with st.spinner("Processing..."):
        print(st.session_state)
        documents = read_text_from_file('./data')
    st.success("Processing complete!")


# Handle newly uploaded files
if uploaded_files:
    if 'uploaded_filenames' not in st.session_state:
        st.session_state['uploaded_filenames'] = []

    new_files = [uploaded_file.name for uploaded_file in uploaded_files if uploaded_file.name not in st.session_state['uploaded_filenames']]
    if new_files:
        for new_file in new_files:
            add_chat_message(f"New file {new_file} has been added.")
        st.session_state['uploaded_filenames'].extend(new_files)

    st.sidebar.write("Uploaded files:")
    for uploaded_file in st.session_state['uploaded_filenames']:
        st.sidebar.write(uploaded_file)

# Button to open/close chatbox
chatbox_button = st.button("Chatbox", key="chatbox_button", help="Click to open the chatbox", on_click=lambda: setattr(st.session_state, 'chatbox_open', not st.session_state['chatbox_open']))

# Chatbox functionality
if st.session_state['chatbox_open']:
    st.write('<style>div.chatbox {position: fixed; bottom: 0; right: 0; width: 300px; background: #f1f1f1; border: 1px solid #ccc; padding: 10px; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);}</style>', unsafe_allow_html=True)
    with st.container():
        st.markdown('<div class="chatbox">', unsafe_allow_html=True)
        st.title("Chatbox")

        # Display chat history
        st.subheader("Chat History")
        chat_history = "\n".join(st.session_state['messages'])
        st.text_area("Chat", chat_history, height=300, disabled=True)

        # Text input for new message
        new_message = st.text_input("Enter your message:", key="new_message_input")

        # Add new message to chat history and bot response
        if st.button("Send", key="send_button"):
            if new_message:
                add_chat_message(f"You: {new_message}")
                # Generate a response using the RAG model
                bot_response = rag_model.generate_response(new_message)
                add_chat_message(f"Bot: {bot_response}")
                st.experimental_rerun()
        st.markdown('</div>', unsafe_allow_html=True)



Writing app.py


In [ ]:
!curl ipecho.net/plain

35.247.19.34

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.247.19.34:8501

npx: installed 22 in 3.107s
your url is: https://dark-lines-join.loca.lt
